In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import pickle
import pandas as pd
import os
import time
from utils_evaluation import print_table_ranking_res

from copy import deepcopy
import pandas as pd
from process_dataset import (
    import_law_eth_all_attributes,
    import_compas_score_processed_all_attributes,
    import_german_processed_all_attributes,
)
from utils_evaluation import print_table_ranking_res




It requires running A_006_cfa_all_attributes.py

In [3]:
min_sup_count = 100
    

for dataset_name in ["law_school_all_attributes", "german_all_attributes", "compas-poc_all_attributes"]:
    if dataset_name == "law_school_all_attributes":
        min_sup = 0.005
    if dataset_name == "compas-poc_all_attributes":
        min_sup = 0.016
    if dataset_name == "german_all_attributes":
        min_sup = 0.1

    output_dir = f'results/cfa_results_mitigation_test_all_attributes_{min_sup_count}'

    filename = os.path.join(output_dir, dataset_name, f'{dataset_name}_cfa_results_{min_sup}.pickle')
    if os.path.isfile(filename):
        with open(filename, 'rb') as fp:
            cfa_ranking_results = pickle.load( fp)

        print(dataset_name)
        r = pd.DataFrame(cfa_ranking_results).T
        display(print_table_ranking_res(r).reset_index())

    output_dir = f'results/cfa_results_mitigation_test_all_attributes_{min_sup_count}'

    filename = os.path.join(output_dir, dataset_name, f'{dataset_name}_cfa_time_{min_sup}.pickle')
    if os.path.isfile(filename):
        with open(filename, 'rb') as fp:
            cfa_time_results = pickle.load( fp)
        print([i/60 for i in list(cfa_time_results.values())], '-', list(cfa_time_results.values())[0])


law_school_all_attributes


,index,method,protected attributes,protected values,#posdiv,#negdiv,minΔ,maxΔ,gini,kendall tau,rank gain,rank drop,ndcgLoss
0,"cfa - UGPA, ZFYA, race, sex - <-0.32, <3.1, Am...",cfa$\theta$,"UGPA, ZFYA, race, sex","<-0.32, <3.1, Amerindian, female, <-0.32, <3.1...",0,5,-0.46,0.19,0.08,0.77,12922,-9106,0.01
1,original,original,all,-,54,88,-8.22,3.02,0.12,1.00,0,0,0.00


[105.77847797473272] - 6346.708678483963
german_all_attributes


,index,method,protected attributes,protected values,#posdiv,#negdiv,minΔ,maxΔ,gini,kendall tau,rank gain,rank drop,ndcgLoss
0,"cfa - age, civil_status, credit_history, exist...",cfa$\theta$,"age, civil_status, credit_history, existing_cr...","'100<=X<500', 'all paid', 'for free', 'high qu...",1006,5913,-0.24,0.32,0.21,0.98,648,-239,0.00
1,original,original,all,-,990,6188,-0.59,0.83,0.46,1.00,0,0,-30.22


[16.262015104293823] - 975.7209062576294
compas-poc_all_attributes


,index,method,protected attributes,protected values,#posdiv,#negdiv,minΔ,maxΔ,gini,kendall tau,rank gain,rank drop,ndcgLoss
0,"cfa - age_cat, c_charge_degree, length_of_stay...",cfa$\theta$,"age_cat, c_charge_degree, length_of_stay, race...","0, 0, 0, 1w-3M, F, 0, 0, 0, 1w-3M, M, 0, 0, 0,...",0,5,-0.02,0.02,0.16,0.6,3348,-4106,0.02
1,original,original,all,-,126,109,-0.24,0.23,0.19,1.0,0,0,0.00


[0.38293639024098713] - 22.97618341445923


# Feldman

In [4]:
DONTCARE = "-"

In [5]:
def get_item_value_pairs(itemset): 
    return [(item.split('=')[0], '='.join(item.split('=')[1:])) for item in itemset]

def add_dont_care(ivpairs, sensitive_attributes, DONTCARE):
    ivpairs_dc = deepcopy(ivpairs)
    if len(ivpairs) == len(sensitive_attributes):
        return ivpairs_dc
    for sensitive_attribute in sensitive_attributes:
        if sensitive_attribute not in [item for item, _ in ivpairs]:
            ivpairs_dc.append((sensitive_attribute, DONTCARE))
    return ivpairs_dc

In [6]:
def get_protected_attribute_values_list_for_feldman_from_patterns(patterns, sensitive_attributes):


    itemsets_ivpairs = []
    for itemset in patterns:
        ivpairs = get_item_value_pairs(itemset)
        ivpairs = add_dont_care(ivpairs, sensitive_attributes, DONTCARE)
        assert len(ivpairs) == len(sensitive_attributes), 'Error'
        itemsets_ivpairs.append(ivpairs)

    protected_attributes_list = []
    protected_attribute_values_list = []
    for itemset in itemsets_ivpairs:
        protected_attributes, protected_attribute_values = zip(*itemset)
        protected_attributes_list.append(protected_attributes)
        protected_attribute_values_list.append(protected_attribute_values)
    return protected_attributes_list, protected_attribute_values_list

In [12]:
dataset_name = "german_all_attributes"

In [13]:
if dataset_name == "law_school_all_attributes":
    min_sup = 0.005
if dataset_name == "compas-poc_all_attributes":
    min_sup = 0.016
if dataset_name == "german_all_attributes":
    min_sup = 0.1

In [14]:
if dataset_name == "law_school_all_attributes":
    data, target_name, sensitive_attributes = import_law_eth_all_attributes()
elif dataset_name == "compas-poc_all_attributes":
    (
        data,
        target_name,
        sensitive_attributes,
    ) = import_compas_score_processed_all_attributes()
elif dataset_name == "german_all_attributes":
    (
        data,
        target_name,
        sensitive_attributes,
    ) = import_german_processed_all_attributes()
    
else:
    raise ValueError()

In [16]:
print(dataset_name)

protected_attributes_list = [[sa] for sa in sensitive_attributes] + [
        sensitive_attributes
    ]

attributes = list(data.columns)
attributes.remove(target_name)


from utils_mitigation import data_and_outcome

# target = "inv_rank"
target = "score"
# target = "utility"

df_input = data_and_outcome(data, target_name, target)

# Top-K disadavantaged groups

K=10

from utils_mitigation import get_fpdiv, get_divergent

fp_i = get_fpdiv(df_input, target, min_sup=min_sup)
fp_div = get_divergent(fp_i, t='t_value_outcome')

patterns = fp_div.head(K).itemsets.values

display(fp_div.head(K)[['itemsets', 'support_count', 'd_outcome']])


sensitive_attributes = attributes
protected_attributes_list, protected_attribute_values_list = get_protected_attribute_values_list_for_feldman_from_patterns(patterns, sensitive_attributes)


from feldman_iterative import feldman_iterative




ranking_results_feldman, res_mit_feldman, time_feldman = \
    feldman_iterative(df_input, target, target_name, attributes, protected_attributes_list, protected_attribute_values_list, min_sup=min_sup)


print('Top-K disadavantaged groups')
r = pd.DataFrame(ranking_results_feldman).T
display(print_table_ranking_res(r))

print('Time')
time_feldman.values
print(np.sum(list(time_feldman.values())))

#####

# Disadavantaged groups of size 1

from utils_mitigation import get_fpdiv, get_negatively_divergent

fp_i = get_fpdiv(df_input, target, min_sup=min_sup)
fp_ndiv = get_negatively_divergent(fp_i, t='t_value_outcome')
fp_ndfiv = fp_ndiv.loc[fp_ndiv['t_value_outcome']>2]

patterns = fp_ndiv.loc[fp_ndiv['length']==1].itemsets.values


display(fp_ndiv.loc[fp_ndiv['length']==1][['itemsets']])

sensitive_attributes = attributes
protected_attributes_list, protected_attribute_values_list = get_protected_attribute_values_list_for_feldman_from_patterns(patterns, sensitive_attributes)


from feldman_iterative import feldman_iterative

ranking_results_feldman, res_mit_feldman, time_feldman = \
    feldman_iterative(df_input, target, target_name, attributes, protected_attributes_list, protected_attribute_values_list, min_sup=min_sup)


print('Disadavantaged groups of size 1')
r = pd.DataFrame(ranking_results_feldman).T
display(print_table_ranking_res(r))

print('Time')
print(np.sum(list(time_feldman.values())))

german_all_attributes


,itemsets,support_count,d_outcome
15112,"(property_magnitude='real estate', installment...",100.0,-0.586406
14951,"(installment_commitment=4, other_payment_plans...",100.0,-0.578919
12743,"(property_magnitude='real estate', installment...",107.0,-0.576613
13591,"(own_telephone=none, installment_commitment=4,...",104.0,-0.575443
13824,"(foreign_worker=yes, property_magnitude='real ...",104.0,-0.571932
14498,"(installment_commitment=4, other_payment_plans...",102.0,-0.569737
12313,"(installment_commitment=4, num_dependents=1, o...",109.0,-0.566393
9385,"(own_telephone=none, installment_commitment=4,...",121.0,-0.562947
12765,"(installment_commitment=4, num_dependents=1, o...",107.0,-0.560785
10069,"(installment_commitment=4, own_telephone=none,...",118.0,-0.559465


Top-K disadavantaged groups


,method,protected attributes,protected values,#posdiv,#negdiv,minΔ,maxΔ,gini,kendall tau,rank gain,rank drop,ndcgLoss
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, -, -, -, -, -, 4, -, 1, -, none, -, 'real estate', -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, -, -, -, -, -, 4, -, 1, -, none, -, 'real e...",990,6188,-0.59,0.83,0.46,1.0,0,0,-30.22
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, married/wid/sep, -, -, -, -, 4, -, 1, -, none, none, -, -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, married/wid/sep, -, -, -, -, 4, -, 1, -, no...",990,6188,-0.59,0.83,0.46,1.0,0,0,-30.22
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, -, -, -, -, -, 4, -, -, -, none, -, 'real estate', -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, -, -, -, -, -, 4, -, -, -, none, -, 'real e...",990,6188,-0.59,0.83,0.46,1.0,0,0,-30.22
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, married/wid/sep, -, -, -, -, 4, -, -, -, none, none, -, -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, married/wid/sep, -, -, -, -, 4, -, -, -, no...",990,6188,-0.59,0.83,0.46,1.0,0,0,-30.22
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, -, -, -, yes, -, 4, -, -, -, none, -, 'real estate', -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, -, -, -, yes, -, 4, -, -, -, none, -, 'real...",990,6188,-0.59,0.83,0.46,1.0,0,0,-30.22
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, married/wid/sep, -, -, yes, -, 4, -, -, -, none, none, -, -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, married/wid/sep, -, -, yes, -, 4, -, -, -, ...",990,6188,-0.59,0.83,0.46,1.0,0,0,-30.22
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, married/wid/sep, -, -, -, -, 4, -, 1, none, -, none, -, -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, married/wid/sep, -, -, -, -, 4, -, 1, none,...",990,6188,-0.59,0.83,0.46,1.0,0,0,-30.22
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, married/wid/sep, -, -, -, -, 4, -, 1, -, -, none, -, -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, married/wid/sep, -, -, -, -, 4, -, 1, -, -,...",990,6188,-0.59,0.83,0.46,1.0,0,0,-30.22
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_depend

Time
0.07311463356018066


,itemsets
837,(property_magnitude='real estate')
121,(installment_commitment=4)
2428,(job='unskilled resident')
39,(own_telephone=none)
856,(purpose=radio/tv)
149,(civil_status=married/wid/sep)
606,(sex=1)


Disadavantaged groups of size 1


,method,protected attributes,protected values,#posdiv,#negdiv,minΔ,maxΔ,gini,kendall tau,rank gain,rank drop,ndcgLoss
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, -, -, -, -, -, -, -, -, -, -, -, 'real estate', -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, -, -, -, -, -, -, -, -, -, -, -, 'real esta...",705,5933,-0.62,0.78,0.46,0.86,190,-75,-27.91
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, -, -, -, -, -, 4, -, -, -, -, -, -, -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, -, -, -, -, -, 4, -, -, -, -, -, -, -, -, -, -",705,5933,-0.62,0.78,0.46,0.86,190,-75,-27.91
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, -, -, -, -, -, -, 'unskilled resident', -, -, -, -, -, -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, -, -, -, -, -, -, 'unskilled resident', -, ...",631,6214,-0.65,0.72,0.45,0.81,282,-105,-27.47
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, -, -, -, -, -, -, -, -, -, -, none, -, -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, -, -, -, -, -, -, -, -, -, -, none, -, -, -...",371,4648,-0.76,0.85,0.45,0.74,335,-183,-24.76
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, -, -, -, -, -, -, -, -, -, -, -, -, radio/tv, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, -, -, -, -, -, -, -, -, -, -, -, -, radio/t...",464,3503,-0.78,0.83,0.44,0.71,432,-223,-24.00
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, married/wid/sep, -, -, -, -, -, -, -, -, -, -, -, -, -, -, -_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, married/wid/sep, -, -, -, -, -, -, -, -, -,...",305,2821,-0.75,0.87,0.43,0.67,465,-293,-23.54
"feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, -, -, -, -, -, -, -, -, -, -, -, -, -, -, -, 1_1000",feldman,"age, civil_status, credit_history, existing_cr...","-, -, -, -, -, -, -, -, -, -, -, -, -, -, -, -, 1",305,2821,-0.75,0.87,0.43,0.67,465,-293,-23.54
"original - feldman - score : age_civil_status_credit_history_existing_credits_foreign_worker_housing_installment_commitment_job_num_dependents_other_parties_other_payment_plans_own_telephone_property_magnitude_purpose_residence_since_savings_status_sex_-, -, -, -, -, -, -, -, -, -, -, -, -, -, -, -, 1_1000",original,"UGPA, ZFYA, race, sex",-,990,6188,-0.59,0.83,0.46,1.00,0,0,-30.22


Time
0.5083816051483154


# DivRank

In [12]:
dataset_name = "german_all_attributes"

In [13]:
if dataset_name == "law_school_all_attributes":
    min_sup = 0.005
if dataset_name == "compas-poc_all_attributes":
    min_sup = 0.016
if dataset_name == "german_all_attributes":
    min_sup = 0.1

In [15]:
min_sup_count = 100
    

for dataset_name in ["law_school_all_attributes", "german_all_attributes", "compas-poc_all_attributes"]:
    if dataset_name == "law_school_all_attributes":
        min_sup = 0.005
    if dataset_name == "compas-poc_all_attributes":
        min_sup = 0.016
    if dataset_name == "german_all_attributes":
        min_sup = 0.1

    output_dir = f'results/divrank_results_mitigation_test_all_attributes_{min_sup_count}'

    filename = os.path.join(output_dir, dataset_name, f'{dataset_name}_divrank_results_mitigation.pickle')
    print(filename)
    if os.path.isfile(filename):
        with open(filename, 'rb') as fp:
            divrank_ranking_results = pickle.load( fp)

        print(dataset_name)
        r = pd.DataFrame(divrank_ranking_results).T
        display(print_table_ranking_res(r).reset_index())

    output_dir = f'results/divrank_results_mitigation_test_all_attributes_{min_sup_count}'

    filename = os.path.join(output_dir, dataset_name, f'{dataset_name}_divrank_results_mitigation_time.pickle')

    if os.path.isfile(filename):
        with open(filename, 'rb') as fp:
            divrank_time_results = pickle.load( fp)
        print([i/60 for i in list(divrank_time_results.values())], '-', list(divrank_time_results.values())[0])


results/divrank_results_mitigation_test_all_attributes_100/law_school_all_attributes/law_school_all_attributes_divrank_results_mitigation.pickle
law_school_all_attributes


,index,method,protected attributes,protected values,#posdiv,#negdiv,minΔ,maxΔ,gini,kendall tau,rank gain,rank drop,ndcgLoss
0,"original - div-mit - score : sex, race, UGPA, ...",original,"UGPA, ZFYA, race, sex",-,54,88,-8.22,3.02,0.12,1.00,0,0,0.00
1,"div-mit - score : sex, race, UGPA, ZFYA",div-mit,"UGPA, ZFYA, race, sex",-,15,0,-0.25,1.24,0.13,0.79,14316,-3440,0.05


[1.2941299994786581] - 77.64779996871948
results/divrank_results_mitigation_test_all_attributes_100/german_all_attributes/german_all_attributes_divrank_results_mitigation.pickle
german_all_attributes


,index,method,protected attributes,protected values,#posdiv,#negdiv,minΔ,maxΔ,gini,kendall tau,rank gain,rank drop,ndcgLoss
0,"original - div-mit - score : credit_history, p...",original,"age, civil_status, credit_history, existing_cr...",-,990,6188,-0.59,0.83,0.46,1.00,0,0,0.0
1,"div-mit - score : credit_history, purpose, sav...",div-mit,"age, civil_status, credit_history, existing_cr...",-,223,0,-0.09,0.38,0.30,0.52,792,-544,0.1


[17.334574631849925] - 1040.0744779109955
results/divrank_results_mitigation_test_all_attributes_100/compas-poc_all_attributes/compas-poc_all_attributes_divrank_results_mitigation.pickle
compas-poc_all_attributes


,index,method,protected attributes,protected values,#posdiv,#negdiv,minΔ,maxΔ,gini,kendall tau,rank gain,rank drop,ndcgLoss
0,"original - div-mit - score : sex, age_cat, rac...",original,"age_cat, c_charge_degree, length_of_stay, race...",-,126,109,-0.24,0.23,0.19,1.00,0,0,0.00
1,"div-mit - score : sex, age_cat, race, c_charge...",div-mit,"age_cat, c_charge_degree, length_of_stay, race...",-,49,0,-0.02,0.07,0.14,0.62,3503,-1756,0.07


[0.4654091437657674] - 27.924548625946045
